In [ ]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show, ColumnDataSource, save
from bokeh.palettes import viridis, brewer, magma, d3, cividis
from bokeh.models import HoverTool, LinearColorMapper
from bokeh.models.tickers import FixedTicker
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import output_notebook, reset_output, output_file, save

import sys
sys.path.append('../..')

from src.visualization.visualize import *

output_notebook()

## Read in and merge data

## Generate fake data for viz purposes

In [ ]:
n_rows = df.shape[0]
p = 0.25

cols = ['R2B_email_sponsorship_promotion', 'R2B_provide_resources', 'R2B_liason', 'B2R_event_participation',
       'B2R_sponsorship_donation', 'B2R_share_business_information', 'B2R_volunteer', 'B2R_use_RVMS_resources']

for col in cols:
    df[col] = np.random.binomial(n=1, p=p, size=n_rows)
    
df['R2B_score'] = df.filter(like='R2B').apply('sum',axis=1)
df['B2R_score'] = df.filter(like='B2R').apply('sum',axis=1)

low = 0.25
high = 0.75
df['R2B_score_noise'] = df['R2B_score'] + np.random.uniform(low=low, high=high, size=n_rows)
df['B2R_score_noise'] = df['B2R_score'] + np.random.uniform(low=low, high=high, size=n_rows)

## Visualization

In [ ]:

# Hover tool
TOOLTIPS = [
    ("index", "$index"),
    ("Business", "@Business_Name"),
    ('Business to RVMS Score', "@B2R_score"),
    ('RVMS to Business Score', "@R2B_score"),
    ("Industry", "@2017_NAICS_US_Title"),
    ("NAICS Code", "@NAICS_Code"),
]

p = figure(title="Business Engagement Matrix", x_axis_label='Business to RVMS', y_axis_label='RVMS to Business',
          x_range=(0, 6), y_range=(0, 4), tooltips=TOOLTIPS)

# Ticks
p.xaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4, 5])
p.yaxis.ticker = FixedTicker(ticks=[1, 2, 3, 4])

# Grid lines
p.xgrid.ticker = FixedTicker(ticks=[1,2,3,4,5])
p.ygrid.ticker = FixedTicker(ticks=[1,2,3,4])

p.scatter(x='B2R_score_noise', y='R2B_score_noise', fill_color='color', radius='radius', 
          source=source)

In [ ]:
show(p)

In [ ]:
output_file("../../data/output/matrix.html")
save(p)
reset_output()